In [1]:
!pip install -qU oikan


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install -qU oikan


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install -qU pmlb


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import numpy as np
import torch
import time
import tracemalloc
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from pmlb import fetch_data
from oikan import OIKANRegressor, OIKANClassifier

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Determine device (CPU or GPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
def adjust_augmentation_factor(n_samples, augmentation_factor=10, min_augmented_size=1000, max_augmented_size=100000):
    augmented_size = augmentation_factor * n_samples
    if augmented_size < min_augmented_size:
        augmentation_factor = max(augmentation_factor, (min_augmented_size + n_samples - 1) // n_samples)
    elif augmented_size > max_augmented_size:
        augmentation_factor = max(1, max_augmented_size // n_samples)
    return augmentation_factor

In [6]:
def benchmark_regression_dataset(dataset_name, counter, total=20):
    """Benchmark OIKANRegressor on a regression dataset."""
    try:
        # Fetch data
        data = fetch_data(dataset_name)
        X = data.drop('target', axis=1).values
        y = data['target'].values
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Adjust augmentation factor
        augmentation_factor = adjust_augmentation_factor(len(X_train))
        num_augmented_rows = augmentation_factor * len(X_train)
        num_features = X.shape[1]
        
        # Initialize model
        model = OIKANRegressor(augmentation_factor=augmentation_factor, verbose=False)
        
        # Training benchmark
        tracemalloc.start()
        start_time = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start_time
        _, train_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        # Prediction benchmark
        tracemalloc.start()
        start_time = time.time()
        y_pred = model.predict(X_test)
        predict_time = time.time() - start_time
        _, predict_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        # Metrics
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Print progress
        print(f"Regression dataset: {dataset_name} - completed ({counter} / {total}) ✅")
        
        return {
            'dataset': dataset_name,
            'device': device,
            'num_features': num_features,
            'num_augmented_rows': num_augmented_rows,
            'augmentation_factor': augmentation_factor,
            'train_time': train_time,
            'train_memory': train_memory / 10**6,  # Convert to MB
            'predict_time': predict_time,
            'predict_memory': predict_memory / 10**6,  # Convert to MB
            'MSE': mse,
            'R2': r2
        }
    except Exception as e:
        print(f"Error benchmarking regression dataset {dataset_name}: {e}")
        return None

In [7]:
def benchmark_classification_dataset(dataset_name, counter, total=20):
    """Benchmark OIKANClassifier on a classification dataset."""
    try:
        # Fetch data
        data = fetch_data(dataset_name)
        X = data.drop('target', axis=1).values
        y = data['target'].values
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Adjust augmentation factor
        augmentation_factor = adjust_augmentation_factor(len(X_train))
        num_augmented_rows = augmentation_factor * len(X_train)
        num_features = X.shape[1]
        
        # Initialize model
        model = OIKANClassifier(augmentation_factor=augmentation_factor, verbose=False)
        
        # Training benchmark
        tracemalloc.start()
        start_time = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start_time
        _, train_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        # Prediction benchmark
        tracemalloc.start()
        start_time = time.time()
        y_pred = model.predict(X_test)
        predict_time = time.time() - start_time
        _, predict_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        # Metrics
        accuracy = accuracy_score(y_test, y_pred)
        
        # Print progress
        print(f"Classification dataset: {dataset_name} - completed ({counter} / {total}) ✅")
        
        return {
            'dataset': dataset_name,
            'device': device,
            'num_features': num_features,
            'num_augmented_rows': num_augmented_rows,
            'augmentation_factor': augmentation_factor,
            'train_time': train_time,
            'train_memory': train_memory / 10**6,  # Convert to MB
            'predict_time': predict_time,
            'predict_memory': predict_memory / 10**6,  # Convert to MB
            'accuracy': accuracy
        }
    except Exception as e:
        print(f"Error benchmarking classification dataset {dataset_name}: {e}")
        return None

In [8]:
def calculate_summary_stats(df, task):
    """Calculate AVG, BEST, WORST for key metrics."""
    if task == 'regression':
        metrics = ['MSE', 'R2']
        best_worst = {'MSE': ('min', 'max'), 'R2': ('max', 'min')}
    else:  # classification
        metrics = ['accuracy']
        best_worst = {'accuracy': ('max', 'min')}
    
    summary = {}
    for metric in metrics:
        avg = df[metric].mean()
        best = df[metric].agg(best_worst[metric][0])
        worst = df[metric].agg(best_worst[metric][1])
        summary[metric] = {'AVG': avg, 'BEST': best, 'WORST': worst}
    return summary

In [9]:
def save_summary_to_txt(summary, filename):
    """Save summary statistics to a .txt file."""
    with open(filename, 'w') as f:
        for metric, stats in summary.items():
            f.write(f"{metric}:\n")
            for key, value in stats.items():
                f.write(f"  {key}: {value:.4f}\n")
            f.write("\n")

In [10]:
print("Starting OIKAN Advanced Auto-Benchmarking based on PMLB Datasets...\n")

# Regression datasets (20)
regression_datasets = [
    '1027_ESL',
    '1028_SWD',
    '1029_LEV',
    '1030_ERA',
    '1089_USCrime',
    '1096_FacultySalaries',
    '228_elusage',
    '229_pwLinear',
    '230_machine_cpu',
    '485_analcatdata_vehicle',
    '529_pollen',
    '556_analcatdata_apnea2',
    '559_bodyfat',
    '573_cpu_act',
    '574_house_16H',
    '583_fri_c1_1000_50',
    '594_fri_c2_1000_25',
    '596_fri_c2_500_25',
    '597_fri_c3_250_25',
    '606_fri_c4_1000_100'
]

# Classification datasets (20)
classification_datasets = [
    'adult',
    'mushroom',
    'iris',
    'wine_recognition',
    'breast_cancer_wisconsin',
    'tic_tac_toe',
    'analcatdata_authorship',
    'analcatdata_fraud',
    'penguins',
    'monk3',
    'balance_scale',
    'car_evaluation',
    'credit_g',
    'dermatology',
    'glass',
    'heart_c',
    'ionosphere',
    'kr_vs_kp',
    'lymphography',
    'zoo'
]

# Benchmark regression datasets
regression_results = []
for idx, dataset in enumerate(regression_datasets, 1):
    result = benchmark_regression_dataset(dataset, counter=idx, total=len(regression_datasets))
    if result:
        regression_results.append(result)
regression_df = pd.DataFrame(regression_results)

# Benchmark classification datasets
classification_results = []
for idx, dataset in enumerate(classification_datasets, 1):
    result = benchmark_classification_dataset(dataset, counter=idx, total=len(classification_datasets))
    if result:
        classification_results.append(result)
classification_df = pd.DataFrame(classification_results)

Starting OIKAN Advanced Auto-Benchmarking based on PMLB Datasets...



/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.330e+02, tolerance: 1.157e+01
  model = cd_fast.enet_coordinate_descent(


Regression dataset: 1027_ESL - completed (1 / 20) ✅

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.644e+02, tolerance: 1.226e+01
  model = cd_fast.enet_coordinate_descent(


Regression dataset: 1028_SWD - completed (2 / 20) ✅


...

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.773e+02, tolerance: 1.602e+00
  model = cd_fast.enet_coordinate_descent(


/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.206e+02, tolerance: 1.185e+00
  model = cd_fast.enet_coordinate_descent(


/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.112e+02, tolerance: 2.702e+00
  model = cd_fast.enet_coordinate_descent(


Classification dataset: lymphography - completed (19 / 20) ✅


Error benchmarking classification dataset zoo: Dataset not found in PMLB.


In [11]:
# Calculate and save summary statistics
if not regression_df.empty:
    regression_summary = calculate_summary_stats(regression_df, 'regression')
    save_summary_to_txt(regression_summary, 'regression_summary.txt')
if not classification_df.empty:
    classification_summary = calculate_summary_stats(classification_df, 'classification')
    save_summary_to_txt(classification_summary, 'classification_summary.txt')

In [12]:
# Display results
print("\nRegression Benchmark Results:")
print(regression_df.to_string(index=False))

print("\nClassification Benchmark Results:")
print(classification_df.to_string(index=False))


Regression Benchmark Results:


                dataset device  num_features  num_augmented_rows  augmentation_factor  train_time  train_memory  predict_time  predict_memory          MSE         R2
               1027_ESL    cpu             4                3900                   10   17.618232     35.434843      0.000926        0.024442 2.667013e-01   0.830337
               1028_SWD    cpu            10                8000                   10    7.720951     24.126501      0.005190        0.085729 4.568245e-01   0.298246
               1029_LEV    cpu             4                8000                   10    7.095260      9.396125      0.002876        0.053922 4.846374e-01   0.432243
               1030_ERA    cpu             4                8000                   10    7.130144      9.393130      0.002878        0.068443 2.617298e+00   0.367802
           1089_USCrime    cpu            13                1036                   28    0.770772      4.551681      0.002154        0.011020 5.156592e+02   0.574244
   1

In [13]:
# Save results to CSV files
regression_df.to_csv('regression_benchmark_results.csv', index=False)
classification_df.to_csv('classification_benchmark_results.csv', index=False)

print("\nBenchmarking completed. Results saved to 'regression_benchmark_results.csv' and 'classification_benchmark_results.csv'.")
print("Summary statistics saved to 'regression_summary.txt' and 'classification_summary.txt'.")


Benchmarking completed. Results saved to 'regression_benchmark_results.csv' and 'classification_benchmark_results.csv'.
Summary statistics saved to 'regression_summary.txt' and 'classification_summary.txt'.
